
<h1 style="font-family:verdana;font-size:300%;text-align:center;background-color:#f2f2f2;color:#0d0d0d">AMMI NLP - Review sessions</h1>

<h1 style="font-family:verdana;font-size:180%;text-align:Center;color:#993333"> Lab 3: n-gram models </h1>

**Big thanks to Amr Khalifa who improved this lab and made it to a Jupyter Notebook!**

In [4]:
import io, sys, math, re
from collections import defaultdict
import numpy as np

In [5]:
from google.colab import drive
drive.mount("drive")

Mounted at drive


In [6]:
# data_loader
def load_data(filename):
    '''
    parameters:
    filename (string): datafile
    
    Returns:
    data (list of lists): each list is a sentence of the text 
    vocab (dictionary): {word: no of times it appears in the text}
    '''
    fin = io.open(filename, 'r', encoding='utf-8')
    data = []
    vocab = defaultdict(lambda:0)
    for line in fin:
        sentence = line.split()
        data.append(sentence)
        for word in sentence:
            vocab[word] += 1
    return data, vocab

In [7]:

print("load training set..")
print("\n")
train_data, vocab = load_data("drive/MyDrive/nlp_week1_lab3_train1.txt")
print(train_data[0])
print("\n")
print("how :",vocab['how'])
print("load validation set")
valid_data, _ = load_data("drive/MyDrive/nlp_week1_lab3_valid1.txt")


load training set..


['<s>', 'my', 'fathers', "don't", 'speak', 'dutch.', '</s>']


how : 107
load validation set


In [9]:
def remove_rare_words(data, vocab, mincount = 1):
    '''
    Parameters:
    data (list of lists): each list is a sentence of the text 
    vocab (dictionary): {word: no of times it appears in the text}
    mincount(int): the minimum count 
    
    Returns: 
    data_with_unk(list of lists): data after replacing rare words with <unk> token
    '''
    # replace words in data that are not in the vocab 
    # or have a count that is below mincount
    data_with_unk = []

    ## FILL CODE
    for sentence in data:
      s = []
      for idx,word in enumerate(sentence):
        if word not in vocab or vocab[word]< mincount:
          s.append("<unk>")
        else:
          s.append(word)
      data_with_unk.append(s)      
    
    return data_with_unk

In [151]:
print("remove rare words")

train_data = remove_rare_words(train_data, vocab, mincount = 2)
valid_data = remove_rare_words(valid_data, vocab, mincount = 1)
#train_data
print(train_data[0])



remove rare words
['<s>', 'my', '<unk>', "don't", 'speak', '<unk>', '</s>']


In [152]:
def build_ngram(data, n):
    '''
    Parameters:
    data (list of lists): each list is a sentence of the text 
    n (int): size of the n-gram
    
    Returns:
    prob (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    '''
    total_number_words = 0
    counts = defaultdict(lambda: defaultdict(lambda: 0.0))
    def _build_ngram(data,n):
      counts = defaultdict(lambda: defaultdict(lambda: 0.0))
      for sentence in data:
          sentence = tuple(sentence)
          ## FILL CODE
          # dict can be indexed by tuples
          # store in the same dict all the ngrams
          # by using the context as a key and the word as a value
          for i in range(0,len(sentence)-(n)):
            ngram = sentence[i:i+(n-1)]
            word = sentence[i+n] if n >1 else sentence[i]
            counts[ngram][word]+=1

      prob = defaultdict(lambda: defaultdict(lambda: 0.0))
      # Build the probabilities from the counts
      # Be careful with how you normalize!

      for context in counts.keys():
      ## FILL CODE
      # p(w | context) = count(context, w)/ count(context)
        count_context = sum([counts[context][w] for w in counts[context].keys()])
        for word in counts[context]:
          prob[context][word] = counts[context][word]/count_context  
      return prob

    prob = defaultdict(lambda: defaultdict(lambda: 0.0))
    for i in range(1,n+1):
      prob = {**prob, **_build_ngram(data,i)}


    
    return prob

In [273]:
# RUN TO BUILD NGRAM MODEL

n = 10
print("build ngram model with n = ", n)
model = build_ngram(train_data, n)


build ngram model with n =  10


Here, implement a recursive function over shorter and shorter context to compute a "stupid backoff model". An interpolation model can also be implemented this way.

In [309]:
def get_prob(model, context, w):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    } 
    context (list of strings): a sentence
    w(string): the word we need to find it's probability given the context
    
    Retunrs:
    prob(float): probability of this word given the context 
    '''

    # code a recursive function over 
    # smaller and smaller context
    # to compute the backoff model
    
    ## FILL CODE
    
    if len(context)==0 and model[context][w] ==0:
      return model[()]["<unk>"]
    if context in model and model[context][w] > 0 :

      return model[context][w]
    
    #new_model = build_ngram(data,len(context) -1)

    return 0.4 *get_prob(model,context[1:],w) 

   


In [310]:
def perplexity(model, data, n):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    } 
    data (list of lists): each list is a sentence of the text
    n(int): size of the n-gram
    
    Retunrs:
    perp(float): the perplexity of the model 
    '''
    ## FILL CODE
    overall_perplexity = 0
    for sentence in data:
      sentence_perplexity = 1
      T = len(sentence)
      for i in range(len(sentence)-n):
        context = tuple(sentence)[i:i+n-1]
        w= sentence[i+n]
        prob = get_prob(model,context,w)
        sentence_perplexity *= prob**(-1/T)
        overall_perplexity+= sentence_perplexity


      

    return overall_perplexity/len(data)

In [311]:
# COMPUTE PERPLEXITY ON VALIDATION SET

print("The perplexity is", perplexity(model, valid_data, n=n))

The perplexity is 2046.4855604190755


In [312]:
def get_proba_distrib(model, context):
    ## need to get the the words after the context and their probability of appearance
    ## after this context 
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    context (list of strings): the sentence we need to find the words after it and 
    thier probabilites
    
    Retunrs:
    words_and_probs(dic): {word: probability of word given context}
    
    '''
    # code a recursive function over context
    # to find the longest available ngram
    
    ## FILL CODE
    
    for i in (range(len(context))):
      #print (context[i:])
      if context[i:] in model:
        return model[context[i:]]
    return model[()]    
    

In [325]:
def generate(model):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    
    Retunrs:
    sentence (list of strings): a sentence sampled according to the language model. 
    

    '''
    # generate a sentence. A sentence starts with a <s> and ends with a </s>
    # Possiblly a use function is:
    # np.random.choice(x, 1, p = y)

    # where x is a list of things to sample from
    # and y is a list of probability (of the same length as x)
    sentence = ["<s>"]
    n =10
    #print (model[("<s>")])
    #print (len(model[tuple(sentence)].values()))
    while sentence[-1] != "</s>" and len(sentence)<10:
        ## FILL CODE
        #print (tuple(sentence)[-1:])
        context= tuple(sentence)[-(n-1):]
        #print(context)
        dist = get_proba_distrib(model,context)
        probs = list(dist.values())
        words = list(dist.keys())
        #print (words)
        #print ()
        word = np.random.choice(words,1,p= probs)[0]
        if word !="<unk>":
          sentence.append(word)
        #print (sentence)
        
    return sentence

In [314]:
# GENERATE A SENTENCE FROM THE MODEL

print("Generated sentence: ",generate(model))

Generated sentence:  ['<s>', 'and', 'brought', 'into', 'police', 'the', '</s>']


Once you are done implementing the model, evaluation and generation code, you can try changing the value of `n`, and play with a larger training set (`train2.txt` and `valid2.txt`). You can also try to implement an interpolation model.

In [285]:

print("load training set 2..")
print("\n")
train_data2, vocab = load_data("drive/MyDrive/nlp_week1_lab3_train2.txt")
print(train_data2[0])
print("\n")
print("how :",vocab['how'])
print("load validation set 2")
valid_data2, _ = load_data("drive/MyDrive/nlp_week1_lab3_valid2.txt")


load training set 2..


['<s>', 'i', 'liked', 'your', 'idea', 'and', 'adopted', 'it', '.', '</s>']


how : 3195
load validation set 2


In [315]:
n = 3

In [317]:
model = build_ngram(train_data2, n)

In [318]:
perplexity(model,valid_data2, n)

203.12509359556086

In [331]:
generate(model)

['<s>', 'is', 'a', 'yen', 'over', 'matter', 'much', '</s>']